In [ ]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem

In [ ]:

consumer = KafkaConsumer(
    'test',
     bootstrap_servers=['54.160.243.136:9092'], #add your IP here
    value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
for c in consumer:
    print(c.value)

In [ ]:
s3 = S3FileSystem()

In [ ]:
for count, i in enumerate(consumer):
    with s3.open("s3://kafka-stock-marketproject-json-bucket/stock_market_{}.json".format(count), 'w') as file:
        json.dump(i.value, file)    

In [ ]:
import s3fs
import json

# Create the S3 filesystem object with region and credentials
s3 = s3fs.S3FileSystem(
    key='**',
    secret='**',
    client_kwargs={'region_name': 'us-east-1'}  # Replace with your S3 bucket's region
)

# Loop to write messages from Kafka consumer to S3
for count, i in enumerate(consumer):
    file_path = f"s3://kafka-stock-marketproject-json-bucket/stock_market_{count}.json"
    with s3.open(file_path, 'w') as file:
        json.dump(i.value, file)


In [ ]:
from kafka import KafkaConsumer
import s3fs
import json

# Kafka consumer setup
consumer = KafkaConsumer(
    'test2',
    bootstrap_servers=['54.160.243.136:9092'],
    value_deserializer=lambda x: json.loads(x.decode('utf-8')) if x else None,  # Handle empty messages
)

# S3 setup
s3 = s3fs.S3FileSystem(
    key='**',
    secret='**',
    client_kwargs={'region_name': 'us-east-1'}
)

# Loop to write messages to S3
for count, record in enumerate(consumer):
    try:
        if record.value:  # Ensure the message has a value
            file_path = f"s3://kafka-stock-marketproject-json-bucket/stock_market_{count}.json"
            with s3.open(file_path, 'w') as file:
                json.dump(record.value, file)  # Write valid JSON to S3
            print(f"Message {count} written to {file_path}")
        else:
            print(f"Skipping empty message at count {count}")
    except json.JSONDecodeError as e:
        print(f"JSON decode error at message {count}: {e}")
    except Exception as e:
        print(f"Error processing message {count}: {e}")
